# Day 7 - Lab 2: Agent Interoperability with A2A Protocol (Solution)

**Objective:** To provide students with hands-on experience implementing the A2A Protocol, enabling them to build two distinct agents that can discover and communicate with each other in a standardized way.

**Introduction:**
This solution notebook provides the complete code for the A2A Protocol lab. It includes the full implementations for both the Responder and Requester agents, and demonstrates how to wrap the A2A client into a LangChain tool for use by a higher-level reasoning agent.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

In [1]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('a2a')

from utils import setup_llm_client, save_artifact
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

a2a not found, installing...


2025-09-21 22:53:14,765 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Implementing the Responder Agent

**Explanation:**
This script creates the 'server' agent. 
1.  We define a standard Python function `calculate_task_complexity`.
2.  The `Service` object from the A2A SDK wraps this function, exposing its signature (name, arguments, types) as a discoverable service.
3.  The `Responder` is the main agent process. We give it a unique name for discovery and register our service with it.
4.  `responder.start()` begins listening on a local port for incoming A2A protocol messages. The `while True` loop keeps the script running to serve requests.

In [2]:
responder_code = """# a2a_responder.py
import time
from a2a_protocol import Responder, Service

# 1. Define the function that will be our service.
def calculate_task_complexity(steps: int, priority: int) -> str:
    '''Calculates a simple complexity score based on steps and priority.'''
    if not isinstance(steps, int) or not isinstance(priority, int):
        return 'Error: Both steps and priority must be integers.'
    complexity = steps * priority
    return f'The calculated complexity score is {complexity}.'

def main():
    print("Initializing Responder Agent...")
    
    # 2. Create a Service object from the function.
    complexity_service = Service(name="calculate_task_complexity", func=calculate_task_complexity)
    
    # 3. Instantiate the Responder.
    responder = Responder(name="ComplexityCalculatorAgent", services=[complexity_service])
    
    try:
        # 4. Start the responder.
        responder.start()
        print(f"Responder '{responder.name}' started at {responder.address}")
        print("Waiting for requests... (Press Ctrl+C to stop)")
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nShutting down responder...")
        responder.stop()

if __name__ == "__main__":
    main()
"""

save_artifact(responder_code, "a2a_responder.py")
print("Saved 'a2a_responder.py'. Run it in a separate terminal.")

Saved 'a2a_responder.py'. Run it in a separate terminal.


### Challenge 2 (Intermediate): Implementing the Requester Agent

**Explanation:**
This script creates the 'client' agent.
1.  `Requester()` creates our client agent.
2.  `requester.discover_responders()` sends out a broadcast on the local network to find any running A2A Responders.
3.  `responder_proxy.discover_services()` connects to a specific responder and asks for its list of available services.
4.  `responder_proxy.invoke(...)` calls the specific service by name, passing the arguments as keyword arguments. The A2A protocol handles the serialization, network communication, and response deserialization.

In [3]:
requester_code = """# a2a_requester.py
import asyncio
from a2a_protocol import Requester

async def main():
    print("Initializing Requester Agent...")
    # 1. Instantiate the Requester
    requester = Requester()

    print("Discovering responders on the network...")
    # 2. Discover responders. This may take a few seconds.
    responders = await requester.discover_responders(timeout=5)

    if not responders:
        print("No responders found. Is the a2a_responder.py script running?")
        return

    # Connect to the first responder found
    responder_proxy = responders[0]
    print(f"Connected to responder: {responder_proxy.name}")

    # 3. Discover the services offered by the responder.
    services = await responder_proxy.discover_services()
    print(f"Discovered services: {services}")

    if 'calculate_task_complexity' in services:
        print("\nInvoking 'calculate_task_complexity' service...")
        # 4. Invoke the service with keyword arguments.
        result = await responder_proxy.invoke('calculate_task_complexity', steps=10, priority=3)
        print(f"Service responded with: {result}")
    else:
        print("The required service was not found.")

if __name__ == "__main__":
    asyncio.run(main())
"""

save_artifact(requester_code, "a2a_requester.py")
print("Saved 'a2a_requester.py'. Run it in a second terminal while the responder is running.")

Saved 'a2a_requester.py'. Run it in a second terminal while the responder is running.


### Challenge 3 (Advanced): Integrating A2A as a LangChain Tool

**Explanation:**
This is the most powerful part of the lab. We abstract the low-level A2A communication behind a simple LangChain tool. 
1.  The `@tool` decorator turns our `get_task_complexity` function into a tool the LangChain agent can see.
2.  The docstring `"Calculates the complexity of a task..."` is critical. This is the *only* information the LLM has about what the tool does. A good docstring is essential for the agent to know when to use the tool.
3.  When the `AgentExecutor` is invoked with a natural language query, the LLM reasons that it needs to calculate complexity, sees our tool is the best fit, and invokes it. 
4.  Our tool's code then runs, performing the A2A communication with the Responder agent to get the final answer, which is then passed back to the LLM to be formatted for the user.

In [4]:
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from a2a_protocol import Requester
import asyncio

llm = ChatOpenAI(model=model_name)

@tool
async def get_task_complexity(steps: int, priority: int) -> str:
    """Calculates the complexity of a task based on the number of steps and its priority level. Use this for any questions about task complexity."""
    requester = Requester()
    responders = await requester.discover_responders(timeout=5)
    if not responders:
        return "Error: Could not find the ComplexityCalculatorAgent."
    
    responder_proxy = responders[0]
    try:
        result = await responder_proxy.invoke('calculate_task_complexity', steps=steps, priority=priority)
        return str(result)
    except Exception as e:
        return f"Error invoking service: {e}"

# Create the LangChain agent
tools = [get_task_complexity]
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# This function is needed to run the async invoke in a sync notebook environment
async def run_agent_query():
    print("--- Invoking LangChain Agent with A2A Tool ---")
    # NOTE: Ensure your a2a_responder.py is running in a separate terminal before executing this cell.
    result = await agent_executor.ainvoke({"input": "How complex is a task with 8 steps and a priority level of 5?"})
    print(f"\nFinal Answer: {result['output']}")

# Run the async function
# asyncio.run(run_agent_query())

ImportError: cannot import name 'Requester' from 'a2a_protocol' (/Users/agaleana/repos/AG-AISOFTDEV/.venv/lib/python3.11/site-packages/a2a_protocol/__init__.py)

## Lab Conclusion

Excellent work! You have successfully implemented the Agent2Agent protocol, creating two distinct agents that can communicate in a standardized way. More importantly, you integrated this low-level communication into a high-level LangChain agent, demonstrating how specialized, protocol-driven agents can be exposed as tools for more general-purpose reasoning agents. This is a key architectural pattern for building complex, interoperable multi-agent systems.